In [2]:
import pandas as pd, numpy as np
import requests
import os
import json
import csv
import datetime
import dateutil.parser
import unicodedata
import time

Computing livedata of twitter about Luzern

In [3]:
def auth():
    return os.getenv('TOKEN')


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_time, end_time, max_results=300):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword + ' -is:retweet -is:reply',
                    'start_time': start_time,
                    'end_time': end_time,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def response_to_df(json_response):
    # response containes two parts : the tweets and the users
    tweets = pd.DataFrame(json_response['data'])
    users = pd.DataFrame(json_response['includes']['users'])
    # enrich the tweets with the username of the author
    id_name_dict = pd.Series(users.username.values, index=users.id).to_dict()
    tweets['username'] = tweets.author_id.map(id_name_dict)
    return tweets

No twitter developer account at the moment and not further worked on it yet.